In [1]:
# https://www.kaggle.com/kinguistics/classifying-news-headlines-with-scikit-learn/notebook
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import pickle
#import sklearn.externals
#import joblib

#from joblib import dump, load
#from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
#from wordcloud import ImageColorGenerator, STOPWORDS, WordCloud

In [2]:
# read to dataframe
news = pd.read_json("news_v02.json")

print("title count:", len(news))
print(news.groupby(["topic"]).count())
news.sample(10)

title count: 59322
                title
topic                
business         9887
entertainment    9887
health           9887
politics         9887
science & tech   9887
travel           9887


,title,topic
1276,Colorado makes nearly $2 million in pot taxes,business
18312,Meet the 'Chicken From Hell'—Scientists Discov...,science & tech
46847,Republican Strategy On Health Care Bill In Flu...,politics
15473,"Report: US, Toyota may reach settlement in probe",science & tech
59090,Most Expensive Cruise: Sailing Costs $1.5 Million,travel
51408,8 Quirky Towns Where You're Free To Be You And Me,travel
58673,Motel 6 Turns 50 (PHOTOS),travel
32252,Weld County kids will 'Kick Butts' on Wednesday,health
46905,Trump Is Setting The Stage To Fire Mueller,politics
9856,Search continues for debris that could possibl...,business


In [3]:
def normalize_text(s):
    s = s.lower()
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W\s',' ',s)
    
    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    return s

news['text'] = [normalize_text(s) for s in news['title']]
print("done normalizing text")
news.sample(10)

done normalizing text


,title,topic,text
8209,Court reverses ruling on swipe fees in favor o...,business,court reverses ruling on swipe fees in favor o...
34940,"Volusia slips in county health rankings, while...",health,volusia slips in county health rankings while ...
23855,Tina Fey shoots down Mean Girls reunion movie ...,entertainment,tina fey shoots down mean girls reunion movie ...
56902,5 Amsterdam Coffeeshops Not To Miss,travel,5 amsterdam coffeeshops not to miss
35048,"Jim Kelly battling cancer, family by his side ...",health,jim kelly battling cancer family by his side i...
3145,GM aware of faulty ignition switch problem as ...,business,gm aware of faulty ignition switch problem as ...
30932,BP reading above normal may increase stroke ri...,health,bp reading above normal may increase stroke ri...
50212,Going To A National Park Anytime Soon? Don't B...,travel,going to a national park anytime soon don't be...
43878,Money Is Politics,politics,money is politics
48073,"A Year After Orlando Shooting, Trump Still Usi...",politics,a year after orlando shooting trump still usin...


In [4]:
# pull the data into vectors
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(news['text'])
# persist vectorizer
with open("pkl-vectorizer.pkl", 'wb') as file :
    pickle.dump(vectorizer, file)
#joblib.dump(vectorizer, "vectorizer.joblib")
#
#vocab = vectorizer.vocabulary_
#print(vocab)

encoder = LabelEncoder()
y = encoder.fit_transform(news['topic'])

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
#print(x_test.shape)

print("DONE vectorizing text")

DONE vectorizing text


In [5]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

# save model
with open("model.pkl", 'wb') as model_file :
    pickle.dump(nb, model_file)
#joblib.dump(nb, "mnb-news_gCloud.joblib")

# load model
'''nb_load = joblib.load("mnb-news_gCloud.joblib")

print(nb_load.score(x_test, y_test))
'''
print("\nDONE!")


DONE!


In [ ]:
# wordcloud
text = [key for key in vocab]
text = " ".join(text)
stop = set(STOPWORDS)
stop.add("will")
wordcloud = WordCloud(stopwords = stop, background_color = "white").generate(text)

plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()